**Due**:Tuesday, June 12, 2024 11:59 PM EST

**Author**:Alejandro M. Ouslan


# Use the data in **WAGE1** in this excercise.
- Find the average education level in the sample. What are the lowest and highest years of education? 
    - **Answer**: The mean for education level is $12.56$, the min is $0$ and the max is $18$.

In [1]:
# import libaries
import statsmodels.api as sm
import polars as pl
import pandas as pd
import numpy as np
import cpi
import os 
os.chdir("..")

In [2]:
# Getting descriptive statistics for education level
wage1 = pl.from_pandas(pd.read_stata("data/raw/WAGE1.dta"))
wage1["educ"].mean()

12.562737642585551

- Find the average hourly wage in the sample. Does it seme high or low?
    - **Answer**: The mean for hourly wages is $5.896$

In [3]:
wage1["wage"].mean()

5.896102674787035

- The wage data are reported in 1976 dollars. Using the Internet or a printed source,
find the Consumer Price Index (CPI) for the years 1976 and 2013.
    - **Answer**:the CPI for the years 1976 to 2013 is $409.41\%$
- Use the CPI values from part (iii) to find the average hourly wage in 2013 dollars. Now does the average hourly wage seem reasonable?
    - **Answer**:the average hourly wage in 2013 is $24.14$

In [4]:
cpi.inflate(wage1["wage"].mean(), 1976, to=2013)

24.13951477698354

- How many women are in the sample? How many men?
    - **Answer:** 252

In [5]:
wage1.select(pl.col("female")).sum()

female
i64
252


# The data set in CEOSAL2 contains information on chief executive officers for U.S. corporations. The variable *salary* is annual compensation, in thousands of dollars, and *ceoten* is prior number of years as company CEO.
- Find the average salary and the average tenure in the sample.
    - **Answer:** The average salary is $\$865.86$ and the average tenure is $7.95$ years.

In [6]:
# Getting descriptive statistics for education level
ceosal2 = pl.from_pandas(pd.read_stata("data/raw/CEOSAL2.dta"))
ceosal2.select(pl.col("salary", "ceoten")).mean()

salary,ceoten
f64,f64
865.864407,7.954802


- How many CEOs are in their first year as CEO (that is, ceoten = 0)? What is the longest tenure as a CEO? 
    - **Answer**: There $5$ CEOs are in their first year as CEO. The are $2$ with the longest tenure, which is $37$ years.

In [7]:
ceosal2.select(pl.col("ceoten")).filter(pl.col("ceoten") == 0).count()

ceoten
u32
5


In [8]:
ceosal2.select(pl.col("ceoten")).sort(["ceoten"], descending=True).head(3)

ceoten
i8
37
37
34


- Estimate the simple regression model $$\log{salary} = \Beta _0 + \Beta _1 ceoten + \epsilon $$ and report your results in the usual form. What is the (approximate) predicted percentage increase in salary given one more year as a CEO?
   - **Answer**: There is not sufficient evidence to support a significant relationship between the CEO's tenure and salary.

In [9]:
df = pd.read_stata("data/raw/CEOSAL2.dta")
df["log_salary"] = np.log(df["salary"])
x = sm.add_constant(df["ceoten"])
reg = sm.OLS(endog=df["log_salary"], exog=x, missing='drop')
print(reg.fit().summary())

                            OLS Regression Results                            
Dep. Variable:             log_salary   R-squared:                       0.013
Model:                            OLS   Adj. R-squared:                  0.008
Method:                 Least Squares   F-statistic:                     2.334
Date:                Wed, 12 Jun 2024   Prob (F-statistic):              0.128
Time:                        12:08:05   Log-Likelihood:                -160.84
No. Observations:                 177   AIC:                             325.7
Df Residuals:                     175   BIC:                             332.0
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          6.5055      0.068     95.682      0.0

# Use the data in HPRICE1 to estimate the mode
$$price = \Beta _0 + \Beta _1  sqrft + \Beta _2 bdrms + \epsilon$$
- Write out the results in equation form.

In [10]:
df = pd.read_stata("data/raw/hprice1.dta")
df["log_price"] = np.log(df["price"])
x = sm.add_constant(df[["sqrft", "bdrms"]])
reg = sm.OLS(endog=df["price"], exog=x, missing='drop')
print(reg.fit().summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.632
Model:                            OLS   Adj. R-squared:                  0.623
Method:                 Least Squares   F-statistic:                     72.96
Date:                Wed, 12 Jun 2024   Prob (F-statistic):           3.57e-19
Time:                        12:08:05   Log-Likelihood:                -488.00
No. Observations:                  88   AIC:                             982.0
Df Residuals:                      85   BIC:                             989.4
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        -19.3150     31.047     -0.622      0.5

- What is the estimated increase in price for a house with one more bedroom,
holding square footage constant?
    - **Answer**: The increase in price fora a house with one more bedroom is $15,000$.
- What is the estimated increase in price for a house with an additional bedroom
that is 140 squarefeet in size? Compare this to your answer in part (ii).
    - **Answer**: The increase in price fora a house with one more bedroom is $15,000$.Since this a linerar equation every addition bedroom will alwayse be and increase of $15,000$.
- What percentage of the variation in price is explained by square footage and
number of bedrooms?
    - **Answer**: $63.2\%$ of the variation in price is explained by square footage and number of bedrooms.However, it is important to note that the is a large condition number which mean their is a strong multicollinearity inflating the $R^2$ value.
- The first house in the sample has sqrft = 2,438 and bdrms = 4. Find the predicted
selling price for this house from the OLS regression line
    - **Answer**: $354,600$



In [11]:
reg.fit().params["const"] + \
reg.fit().params["sqrft"] * 2438 + \
reg.fit().params["bdrms"] * 4

354.6052489839995

- The actual selling price of the first house in the sample was $300,000 (so price =
300). Find the residual for this house. Does it suggest that the buyer underpaid or
overpaid for the house?
    - **Answer**: This means that the buyer underpaid for the house.
# Refer to Computer Exercise C2 in Chapter 3 (Question above). Now, use the log of the housing price as the dependent variable: 
$$\log(price) = \Beta _0 + \Beta _1  sqrft + \Beta _2 bdrms + \epsilon$$

- You are interested in estimating and obtaining a confidence interval for the
percentage change in price when a 150-square-foot bedroom is added to a house. In decimal form, this is $\theta _1 = 150 \Beta _1 + \Beta _2$. Use the data in HPRICE1 to estimate $\theta _1$.
    - **Answer**: $0.0858$

In [12]:
reg = sm.OLS(endog=df["log_price"], exog=x, missing='drop')
print(reg.fit().summary())

                            OLS Regression Results                            
Dep. Variable:              log_price   R-squared:                       0.588
Model:                            OLS   Adj. R-squared:                  0.579
Method:                 Least Squares   F-statistic:                     60.73
Date:                Wed, 12 Jun 2024   Prob (F-statistic):           4.17e-17
Time:                        12:08:05   Log-Likelihood:                 19.592
No. Observations:                  88   AIC:                            -33.18
Df Residuals:                      85   BIC:                            -25.75
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          4.7660      0.097     49.112      0.0

In [13]:
theta = 150 * reg.fit().params["sqrft"] + reg.fit().params["bdrms"] 
theta

0.08580134441876913

- Write $\Beta _2$ in terms of $\theta _1$ and $\Beta _2$ and plug this into the log(price) equation.
    - 
- Use part (ii) to obtain a standard error for 𝜃1 and use this standard error to (150bta-thata and beta2-1)
construct a 95% confidence interval.
    - **Answer**: $(150.17, 149.83)$

In [20]:
df["sqrt_bdroom"] = df["sqrft"] - 150 * df["bdrms"]

In [27]:
df["sqrt_bdroom"] = df["sqrft"] - (150 * df["bdrms"])
x = sm.add_constant(df[["sqrt_bdroom", "bdrms"]])
reg = sm.OLS(endog=df["log_price"], exog=x, missing='drop')
print(reg.fit().summary())

                            OLS Regression Results                            
Dep. Variable:              log_price   R-squared:                       0.588
Model:                            OLS   Adj. R-squared:                  0.579
Method:                 Least Squares   F-statistic:                     60.73
Date:                Wed, 12 Jun 2024   Prob (F-statistic):           4.17e-17
Time:                        12:40:35   Log-Likelihood:                 19.592
No. Observations:                  88   AIC:                            -33.18
Df Residuals:                      85   BIC:                            -25.75
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const           4.7660      0.097     49.112      